##### Libray

In [1]:
import numpy as np
import pandas as pd
import json

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# path 설정 logdata로 결로 필요
path = 'C:/Users/user654/Desktop/data1/'

In [2]:
with open(path + 'digitalTwin_data.log', encoding = 'utf8') as f:
    logs = f.read().splitlines()
    
    
print('전체 log data의 길이 : ', len(logs))

416738

### <center>데이터 구조 파악</center>

##### dictionary 형태로 변환
- 현재 logs는 list 형태이며 , 각각의 원소들은 str형태
- str 형태 -> literal_eval이라는 함수를 통해 dictionary형태로 변환 후 , key값을 파악


In [3]:
from ast import literal_eval # string 형태를 dictionary로 변환 함수

dict(literal_eval(logs[0])).keys()

dict_keys(['d', 'dt'])

>- key값은 d와 dt임을 알 수 있다.

##### 어린이집 버스 데이터 ? 

In [73]:
#log 0
print(logs[0])

{"d":[{"bid":"24:0A:C4:4F:25:A8","env":[0.0,0.0,0,0,0,0,0],"dt":"2020-07-21T00:02:41.038+0900","d":[]}],"dt":"2020-07-21T00:02:41.038+0900"}


In [74]:
#log 1
print(logs[1])

{"d":[{"bid":"4C:11:AE:B8:C9:A4","env":[0.0,0.0,0,0,0,0,0],"dt":"2020-07-21T00:02:41.386+0900","d":[]}],"dt":"2020-07-21T00:02:41.386+0900"}


In [75]:
#log 2

print(logs[2]) #데이터가 상당히 많음,, 위의 0, 1과 동일한 형태

{"d":[{"bid":"24:0A:C4:4E:A9:10","env":[29.6,61.4,485,96,14,14,14],"dt":"2020-07-21T00:02:40.796+0900","d":[{"id":"f5:88:ed:00:02:79","type":"b","d":[0,0,0,0,0,0,0,0,1,71,-72],"dt":"2020-07-21T00:02:31.725+0900"},{"id":"d3:dd:d3:5b:88:c2","type":"b","d":[0,0,0,0,0,0,0,0,1,100,-62],"dt":"2020-07-21T00:02:31.745+0900"},{"id":"02:11:23:34:5b:7a","type":"t","d":[-79],"dt":"2020-07-21T00:02:31.755+0900"},{"id":"02:11:23:34:5b:bd","type":"t","d":[-68],"dt":"2020-07-21T00:02:31.805+0900"},{"id":"d3:dd:d3:5b:88:c2","type":"b","d":[0,0,0,0,0,0,0,0,1,100,-75],"dt":"2020-07-21T00:02:31.945+0900"},{"id":"02:11:23:34:5b:76","type":"t","d":[-83],"dt":"2020-07-21T00:02:31.965+0900"},{"id":"e4:13:7d:7b:d7:0a","type":"b","d":[0,0,0,0,0,0,0,0,1,0,-78],"dt":"2020-07-21T00:02:32.025+0900"},{"id":"f5:c3:3a:11:23:17","type":"b","d":[0,0,0,0,0,0,0,0,1,85,-63],"dt":"2020-07-21T00:02:32.195+0900"},{"id":"e4:15:d7:13:40:01","type":"b","d":[0,0,0,0,0,0,30,25,1,94,-70],"dt":"2020-07-21T00:02:32.355+0900"},{"id":"

In [77]:
dict(literal_eval(logs[2])).keys()
#데이터가 많아도 , 위 0, 1과 동일하게, key값은 d와 dt뿐

dict_keys(['d', 'dt'])

In [7]:
#log 3 : 버스데이터

print(logs[3]) #버스데이터로 추정됨

{lng":"000.000000","lat":"00.000000",dt:2020-07-21T00:02:41.678}


##### <font color = 'red'>어린이집 버스 데이터</font>
- log file로 받아올때 약간의 정제가 필요하다고 생각됨
- 아래 key값인 lng는 앞에 큰따음표가 없음
- dt는,  key값이지만 큰따음표가 존재하지 않음

In [63]:
# log file 에서 위경도 표기되지 않은 값도 상당히 많이 존재
all_location_list = [i for i in logs if 'lng' in i]
print(len(all_location_list))

all_location_list[:5] #너무 많으므로 상위 5개만 표시

80316


['{lng":"000.000000","lat":"00.000000",dt:2020-07-21T00:02:41.678}',
 '{lng":"000.000000","lat":"00.000000",dt:2020-07-21T00:02:42.681}',
 '{lng":"000.000000","lat":"00.000000",dt:2020-07-21T00:02:43.683}',
 '{lng":"000.000000","lat":"00.000000",dt:2020-07-21T00:02:44.686}',
 '{lng":"000.000000","lat":"00.000000",dt:2020-07-21T00:02:45.688}']

In [84]:
# log file 에서 , 위치데이터 중,  위경도값이 0 이 아닌 값들 출력

exist_location_list = [i for i in logs if ('lng' in i) and ('000.00' not in i)]


# 7월 21일 06:52 부터 위경도 좌표가 찍히는것을 볼 수 있음
# 그리고 위경도가 변함으로써, 움직이는것 또한 확인할 수 있음
exist_location_list[:5]

['{lng":"127.030795","lat":"37.284315",dt:2020-07-21T00:06:52.525}',
 '{lng":"127.030805","lat":"37.284301",dt:2020-07-21T00:06:53.528}',
 '{lng":"127.030815","lat":"37.284290",dt:2020-07-21T00:06:54.531}',
 '{lng":"127.030898","lat":"37.284315",dt:2020-07-21T00:06:55.534}',
 '{lng":"127.030898","lat":"37.284328",dt:2020-07-21T00:06:57.536}']

In [90]:
# 마지막 5개의 데이터를 보면 , 새벽이지만 위경도가 찍힘
# but, 당연히 움직이지 않는것을 볼 수 있음

exist_location_list[-5:]

['{lng":"127.030948","lat":"37.284410",dt:2020-07-22T00:02:38.271}',
 '{lng":"127.030948","lat":"37.284410",dt:2020-07-22T00:02:40.274}',
 '{lng":"127.030948","lat":"37.284410",dt:2020-07-22T00:02:41.277}',
 '{lng":"127.030948","lat":"37.284410",dt:2020-07-22T00:02:42.280}',
 '{lng":"127.030948","lat":"37.284410",dt:2020-07-22T00:02:43.283}']

In [92]:
print('위경도 데이터 존재하는 비율 : ', len(exist_location_list) / len(all_location_list))
print('위경도 데이터 결측치 비율 : ', 1 - (len(exist_location_list) / len(all_location_list)))

위경도 데이터 존재하는 비율 :  0.8894616265750286
위경도 데이터 결측치 비율 :  0.11053837342497141


##### 전체적인 log 데이터의 수
- 8만건의 버스데이터 (위, 경도)
- 하루 41만건의 데이터

In [83]:
# logs 전체 데이터는 41만건
print('전체 log data 길이 : ', len(logs))
print('그 중 버스 데이터의 개수 ', len(all_location_list))
print(len(all_location_list) / len(logs)) 

#전체 중약 19%가 버스데이터, 나머지인 81%는 밴드 및 태그데이터

전체 log data 길이 :  416738
그 중 버스 데이터의 개수  80316
0.19272540541059371


----

# <center>분석을 위한 데이터프레임으로 변환 </center> 

##### 위에서 불러온 log파일을 데이터프레임으로 변환
- 나중에 불러오기 및 처리에 용이하도록 , csv가 아닌 , type을 유지해주는 pickle 사용

In [11]:
# df = pd.DataFrame({})
# locate_df = pd.DataFrame({})

# for i in tqdm(range(len(logs))):
#     try:
#         df = df.append(literal_eval(logs[i])['d'], ignore_index = True)
    
#     except: # 위경도 포함된 데이터인데, 
#         pass
#         #locate_df = locate_df.append()  , 나중에 추가해줄 필요 있음. 전처리 후 추가

# ##### pickle로 저장        
# #df.to_pickle("band_log_to_df.pkl")

# df

# Start

In [33]:
import pandas as pd

# 사전에 만들어 놓은 pickle 형태의 파일 load
df = pd.read_pickle('band_log_to_df.pkl')

df

,bid,d,dt,env,gwId,gwIp,gwVer,trgIp,trgPort
0,24:0A:C4:4F:25:A8,[],2020-07-21T00:02:41.038+0900,"[0.0, 0.0, 0, 0, 0, 0, 0]",NaN,NaN,NaN,NaN,NaN
1,4C:11:AE:B8:C9:A4,[],2020-07-21T00:02:41.386+0900,"[0.0, 0.0, 0, 0, 0, 0, 0]",NaN,NaN,NaN,NaN,NaN
2,24:0A:C4:4E:A9:10,"[{'id': 'f5:88:ed:00:02:79', 'type': 'b', 'd':...",2020-07-21T00:02:40.796+0900,"[29.6, 61.4, 485, 96, 14, 14, 14]",NaN,NaN,NaN,NaN,NaN
3,24:0A:C4:4E:A8:84,[],2020-07-21T00:02:42.214+0900,"[0.0, 0.0, 0, 0, 0, 0, 0]",NaN,NaN,NaN,NaN,NaN
4,C4:4F:33:25:E7:81,[],2020-07-21T00:02:42.096+0900,"[0.0, 0.0, 0, 0, 0, 0, 0]",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
336188,4C:11:AE:B8:CC:CC,[],2020-07-22T00:02:43.135+0900,"[0.0, 0.0, 0, 0, 0, 0, 0]",NaN,NaN,NaN,NaN,NaN
336189,24:0A:C4:4E:E4:2C,[],2020-07-22T00:02:43.328+0900,"[0.0, 0.0, 0, 0, 0, 0, 0]",NaN,NaN,NaN,NaN,NaN
336190,24:0A:C4:4E:CE:E0,[],2020-07-22T00:02:43.377+0900,"[0.0, 0.0, 0, 0, 0, 0, 0]",NaN,NaN,NaN,NaN,NaN
336191,24:0A:C4:4E:A9:10,"[{'id': '02:11:23:34:5b:bf', 'type': 't', 'd':...",2020-07-22T00:02:42.701+0900,"[29.9, 58.6, 736, 93, 10, 10, 10]",NaN,NaN,NaN,NaN,NaN


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336193 entries, 0 to 336192
Data columns (total 9 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   bid      336193 non-null  object
 1   d        336193 non-null  object
 2   dt       336193 non-null  object
 3   env      336193 non-null  object
 4   gwId     2 non-null       object
 5   gwIp     2 non-null       object
 6   gwVer    2 non-null       object
 7   trgIp    2 non-null       object
 8   trgPort  2 non-null       object
dtypes: object(9)
memory usage: 23.1+ MB


In [13]:
# gwId 결측치 아닌 행이 두개뿐?
# 게이트웨이에 대한 정보가 제대로 안받아져 와지는것 같음

display(df[df['gwId'].notna()])

,bid,d,dt,env,gwId,gwIp,gwVer,trgIp,trgPort
106620,24:0A:C4:4E:CE:08,"[{'id': 'e6:d0:c1:9a:48:c5', 'type': 'b', 'd':...",2020-07-21T07:42:44.024+0900,"[29.6, 60.4, 474, 103, 18, 18, 18]",24:0A:C4:4E:CE:08,192.168.1.168,V0.31,digitaltwin.openit.co.kr,1813
218549,24:0A:C4:4E:CE:08,"[{'id': 'dd:34:01:01:b8:9e', 'type': 't', 'd':...",2020-07-21T15:38:08.608+0900,"[30.0, 57.1, 922, 93, 18, 18, 18]",24:0A:C4:4E:CE:08,192.168.1.168,V0.31,digitaltwin.openit.co.kr,1813


##### gwId,gwIp, gwVer	, trgIp, trgPort  column은 무의미하다고 판단되서 삭제

In [14]:
df.drop(['gwId','gwIp', 'gwVer', 'trgIp', 'trgPort'], axis = 1 , inplace = True)
df

,bid,d,dt,env
0,24:0A:C4:4F:25:A8,[],2020-07-21T00:02:41.038+0900,"[0.0, 0.0, 0, 0, 0, 0, 0]"
1,4C:11:AE:B8:C9:A4,[],2020-07-21T00:02:41.386+0900,"[0.0, 0.0, 0, 0, 0, 0, 0]"
2,24:0A:C4:4E:A9:10,"[{'id': 'f5:88:ed:00:02:79', 'type': 'b', 'd':...",2020-07-21T00:02:40.796+0900,"[29.6, 61.4, 485, 96, 14, 14, 14]"
3,24:0A:C4:4E:A8:84,[],2020-07-21T00:02:42.214+0900,"[0.0, 0.0, 0, 0, 0, 0, 0]"
4,C4:4F:33:25:E7:81,[],2020-07-21T00:02:42.096+0900,"[0.0, 0.0, 0, 0, 0, 0, 0]"
...,...,...,...,...
336188,4C:11:AE:B8:CC:CC,[],2020-07-22T00:02:43.135+0900,"[0.0, 0.0, 0, 0, 0, 0, 0]"
336189,24:0A:C4:4E:E4:2C,[],2020-07-22T00:02:43.328+0900,"[0.0, 0.0, 0, 0, 0, 0, 0]"
336190,24:0A:C4:4E:CE:E0,[],2020-07-22T00:02:43.377+0900,"[0.0, 0.0, 0, 0, 0, 0, 0]"
336191,24:0A:C4:4E:A9:10,"[{'id': '02:11:23:34:5b:bf', 'type': 't', 'd':...",2020-07-22T00:02:42.701+0900,"[29.9, 58.6, 736, 93, 10, 10, 10]"


##### 환경(env) 변수가 결측치인 경우는 모두 제거해주기로 함

In [93]:
import numpy as np

env_df = df[df['env'].apply(lambda x : np.sum(x)) > 0] # 주위환경데이터 모두 0인것은 결측치로 간주하여 제거
env_df = env_df.reset_index(drop = True) # index column 초기화


#RAM 용량 부족으로 삭제
del df

env_df

,bid,d,dt,env,gwId,gwIp,gwVer,trgIp,trgPort
0,24:0A:C4:4E:A9:10,"[{'id': 'f5:88:ed:00:02:79', 'type': 'b', 'd':...",2020-07-21T00:02:40.796+0900,"[29.6, 61.4, 485, 96, 14, 14, 14]",NaN,NaN,NaN,NaN,NaN
1,4C:11:AE:B8:C9:58,"[{'id': '02:11:23:34:5b:76', 'type': 't', 'd':...",2020-07-21T00:02:42.802+0900,"[29.6, 61.4, 486, 97, 14, 14, 14]",NaN,NaN,NaN,NaN,NaN
2,24:0A:C4:4E:E4:38,"[{'id': '02:11:23:34:5b:76', 'type': 't', 'd':...",2020-07-21T00:02:43.561+0900,"[29.6, 61.4, 486, 96, 14, 14, 14]",NaN,NaN,NaN,NaN,NaN
3,24:0A:C4:4E:A4:44,"[{'id': '02:11:23:34:5b:76', 'type': 't', 'd':...",2020-07-21T00:02:44.714+0900,"[29.6, 61.4, 486, 96, 14, 14, 14]",NaN,NaN,NaN,NaN,NaN
4,4C:11:AE:B8:BB:58,"[{'id': '02:11:23:34:5b:99', 'type': 't', 'd':...",2020-07-21T00:02:44.036+0900,"[29.6, 61.4, 486, 96, 14, 14, 14]",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
131207,24:0A:C4:4E:A4:C4,"[{'id': '02:11:23:34:5b:a5', 'type': 't', 'd':...",2020-07-22T00:02:40.498+0900,"[29.9, 58.6, 736, 93, 10, 10, 10]",NaN,NaN,NaN,NaN,NaN
131208,24:0A:C4:4E:E4:54,"[{'id': 'ea:92:a1:d3:24:ed', 'type': 'b', 'd':...",2020-07-22T00:02:42.198+0900,"[29.9, 58.6, 736, 95, 9, 9, 9]",NaN,NaN,NaN,NaN,NaN
131209,C4:4F:33:25:EE:BD,[],2020-07-22T00:02:43.058+0900,"[29.9, 58.6, 736, 95, 9, 9, 9]",NaN,NaN,NaN,NaN,NaN
131210,24:0A:C4:4E:A9:10,"[{'id': '02:11:23:34:5b:bf', 'type': 't', 'd':...",2020-07-22T00:02:42.701+0900,"[29.9, 58.6, 736, 93, 10, 10, 10]",NaN,NaN,NaN,NaN,NaN


> # env
> - 1. 온도 : temp
- 2. 습도 : hum
- 3. 이산화탄소 : co2
- 4. 휘발성유기화합물 : voc
- 5. 미세먼지농도1 : dust1
- 6. 초미세먼지농도2.5 , dust2_5
- 7. 초미세먼지농도10 : dust10
##### 위 데이터프레임에서 list형태인 env Column 값들을 각 Column으로 생성시켜줌

In [16]:
env_df['temp'] = env_df['env'].apply(lambda x : x[0])
env_df['hum'] = env_df['env'].apply(lambda x : x[1])
env_df['co2'] = env_df['env'].apply(lambda x : x[2])
env_df['voc'] = env_df['env'].apply(lambda x : x[3])
env_df['dust1'] = env_df['env'].apply(lambda x : x[4])
env_df['dust2_5'] = env_df['env'].apply(lambda x : x[5])
env_df['dust10'] = env_df['env'].apply(lambda x : x[6])

env_df.drop('env', axis = 1 , inplace = True)

env_df.tail()

,bid,d,dt,temp,hum,co2,voc,dust1,dust2_5,dust10
131207,24:0A:C4:4E:A4:C4,"[{'id': '02:11:23:34:5b:a5', 'type': 't', 'd':...",2020-07-22T00:02:40.498+0900,29.9,58.6,736,93,10,10,10
131208,24:0A:C4:4E:E4:54,"[{'id': 'ea:92:a1:d3:24:ed', 'type': 'b', 'd':...",2020-07-22T00:02:42.198+0900,29.9,58.6,736,95,9,9,9
131209,C4:4F:33:25:EE:BD,[],2020-07-22T00:02:43.058+0900,29.9,58.6,736,95,9,9,9
131210,24:0A:C4:4E:A9:10,"[{'id': '02:11:23:34:5b:bf', 'type': 't', 'd':...",2020-07-22T00:02:42.701+0900,29.9,58.6,736,93,10,10,10
131211,24:0A:C4:4E:A8:B8,"[{'id': 'dd:34:01:02:f9:3e', 'type': 't', 'd':...",2020-07-22T00:02:43.654+0900,29.9,58.6,736,95,9,9,9


##### pickle 형태로 저장

In [18]:
##### env_df.to_pickle('env_df.pkl')

##### 같은 환경 내에 있는 여러개의 밴드데이터들을 각 행으로 만들어주는 과정
- "d" Column
    - type : __t__와 __b__로 나뉨 => tag_df, band_df 두개로 나눠주기로 함
    - "d" Column을 정제하여 아래 셀에서 보이는 것처럼 데이터프레임 생성

In [20]:
from tqdm.notebook import tqdm  #for loop 진행현황 확인 library

band_df = pd.DataFrame({})
tag_df = pd.DataFrame({})

env_col = ['bid','dt','temp','hum','co2','voc','dust1','dust2_5','dust10'] #주위환경 column
concat_df = env_df.loc[:, env_col]

for i in tqdm(range(env_df.shape[0])):

    x = pd.DataFrame(env_df['d'][i])
    for col in env_col:
        x[col] = env_df.loc[i, col]    

    try:  # 데이터 결측치이면 error발생
        band_df = band_df.append(x[x['type'] == 'b'] , ignore_index = True)  #band
        tag_df = tag_df.append(x[x['type'] == 't'] , ignore_index = True)    #tag    
    except:
        pass
    
    
# 최종적으로 주위환경 column 결측치인 데이터 제거
band_df.tail()

,id,type,d,dt,bid,temp,hum,co2,voc,dust1,dust2_5,dust10
3022593,d7:78:31:23:3a:fb,b,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, -75]",2020-07-22T00:02:42.701+0900,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,10
3022594,d3:dd:d3:5b:88:c2,b,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 100, -63]",2020-07-22T00:02:42.701+0900,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,10
3022595,cb:d7:d9:4a:31:4f,b,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 100, -76]",2020-07-22T00:02:42.701+0900,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,10
3022596,c6:8e:85:34:bd:09,b,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 100, -67]",2020-07-22T00:02:42.701+0900,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,10
3022597,ec:50:2e:b5:cf:c4,b,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, -67]",2020-07-22T00:02:42.701+0900,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,10


>- band_df에서 d 열은, [운동모드, 운동타입, 심박수, 걸음수, 칼로리 , 거리, 수면상태지수 , 수면시간, 착용상태, 상태지수, 신호 세기]
>- tag_df에서 d 열은 신호세기를 나타냄

- 운동모드 : mode
- 운동타입 : type
- 심박수 : hr (heart rate)
- 걸음수 : walk
- 칼로리 : kcal
- 거리 : dist
- 수면상태지수 : sl_idx
- 수면시간 : sl_time
- 착용상태 : wear
- 상태지수 : st_idx
- 신호세기 : signal

##### band 데이터 전환

In [21]:
band_df['mode'] = band_df['d'].apply(lambda x : x[0])
#band_df['type'] = band_df['d'].apply(lambda x : x[1])
band_df['hr'] = band_df['d'].apply(lambda x : x[2])
band_df['walk'] = band_df['d'].apply(lambda x : x[3])
band_df['kcal'] = band_df['d'].apply(lambda x : x[4])
band_df['dist'] = band_df['d'].apply(lambda x : x[5])
band_df['sl_idx'] = band_df['d'].apply(lambda x : x[6])
band_df['sl_time'] = band_df['d'].apply(lambda x : x[7])
band_df['wear'] = band_df['d'].apply(lambda x : x[8])
band_df['st_idx'] = band_df['d'].apply(lambda x : x[9])
band_df['signal'] = band_df['d'].apply(lambda x : x[10])


band_df.drop('d', axis = 1 , inplace = True)
band_df

,id,type,dt,bid,temp,hum,co2,voc,dust1,dust2_5,...,mode,hr,walk,kcal,dist,sl_idx,sl_time,wear,st_idx,signal
0,f5:88:ed:00:02:79,b,2020-07-21T00:02:40.796+0900,24:0A:C4:4E:A9:10,29.6,61.4,485,96,14,14,...,0,0,0,0,0,0,0,1,71,-72
1,d3:dd:d3:5b:88:c2,b,2020-07-21T00:02:40.796+0900,24:0A:C4:4E:A9:10,29.6,61.4,485,96,14,14,...,0,0,0,0,0,0,0,1,100,-62
2,d3:dd:d3:5b:88:c2,b,2020-07-21T00:02:40.796+0900,24:0A:C4:4E:A9:10,29.6,61.4,485,96,14,14,...,0,0,0,0,0,0,0,1,100,-75
3,e4:13:7d:7b:d7:0a,b,2020-07-21T00:02:40.796+0900,24:0A:C4:4E:A9:10,29.6,61.4,485,96,14,14,...,0,0,0,0,0,0,0,1,0,-78
4,f5:c3:3a:11:23:17,b,2020-07-21T00:02:40.796+0900,24:0A:C4:4E:A9:10,29.6,61.4,485,96,14,14,...,0,0,0,0,0,0,0,1,85,-63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3022593,d7:78:31:23:3a:fb,b,2020-07-22T00:02:42.701+0900,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,...,0,0,0,0,0,0,0,1,0,-75
3022594,d3:dd:d3:5b:88:c2,b,2020-07-22T00:02:42.701+0900,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,...,0,0,0,0,0,0,0,1,100,-63
3022595,cb:d7:d9:4a:31:4f,b,2020-07-22T00:02:42.701+0900,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,...,0,0,0,0,0,0,0,1,100,-76
3022596,c6:8e:85:34:bd:09,b,2020-07-22T00:02:42.701+0900,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,...,0,0,0,0,0,0,0,1,100,-67


##### tag 데이터프레임의 값 전처리

In [23]:
tag_df['d'] = tag_df['d'].apply(lambda x : x[0])
tag_df

,id,type,d,dt,bid,temp,hum,co2,voc,dust1,dust2_5,dust10
0,02:11:23:34:5b:7a,t,-79,2020-07-21T00:02:40.796+0900,24:0A:C4:4E:A9:10,29.6,61.4,485,96,14,14,14
1,02:11:23:34:5b:bd,t,-68,2020-07-21T00:02:40.796+0900,24:0A:C4:4E:A9:10,29.6,61.4,485,96,14,14,14
2,02:11:23:34:5b:76,t,-83,2020-07-21T00:02:40.796+0900,24:0A:C4:4E:A9:10,29.6,61.4,485,96,14,14,14
3,02:11:23:34:5b:c2,t,-83,2020-07-21T00:02:40.796+0900,24:0A:C4:4E:A9:10,29.6,61.4,485,96,14,14,14
4,02:11:23:34:5b:a4,t,-77,2020-07-21T00:02:40.796+0900,24:0A:C4:4E:A9:10,29.6,61.4,485,96,14,14,14
...,...,...,...,...,...,...,...,...,...,...,...,...
3591930,02:11:23:34:5b:d3,t,-87,2020-07-22T00:02:42.701+0900,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,10
3591931,02:11:23:34:5b:d3,t,-86,2020-07-22T00:02:42.701+0900,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,10
3591932,02:11:23:34:5b:b7,t,-69,2020-07-22T00:02:42.701+0900,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,10
3591933,dd:34:01:02:f9:3e,t,-93,2020-07-22T00:02:43.654+0900,24:0A:C4:4E:A8:B8,29.9,58.6,736,95,9,9,9


##### dt를 datetime형으로 형변환


In [24]:
band_df['dt'] = pd.to_datetime(band_df['dt'])
tag_df['dt'] = pd.to_datetime(tag_df['dt'])

##### dt에서 시간만 따로 추출

In [25]:
band_df['hour'] = band_df['dt'].apply(lambda x : x.hour)
tag_df['hour'] = tag_df['dt'].apply(lambda x : x.hour)

band_df.tail()

,id,type,dt,bid,temp,hum,co2,voc,dust1,dust2_5,...,hr,walk,kcal,dist,sl_idx,sl_time,wear,st_idx,signal,hour
3022593,d7:78:31:23:3a:fb,b,2020-07-22 00:02:42.701000+09:00,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,...,0,0,0,0,0,0,1,0,-75,0
3022594,d3:dd:d3:5b:88:c2,b,2020-07-22 00:02:42.701000+09:00,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,...,0,0,0,0,0,0,1,100,-63,0
3022595,cb:d7:d9:4a:31:4f,b,2020-07-22 00:02:42.701000+09:00,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,...,0,0,0,0,0,0,1,100,-76,0
3022596,c6:8e:85:34:bd:09,b,2020-07-22 00:02:42.701000+09:00,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,...,0,0,0,0,0,0,1,100,-67,0
3022597,ec:50:2e:b5:cf:c4,b,2020-07-22 00:02:42.701000+09:00,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,...,0,0,0,0,0,0,1,0,-67,0


##### 위 tail에서 1,2행에서 볼 수 있듯이 , 같은 id로 시간이 중복되는 경우가 상당히 많음
- 그래서 , id와 dt가 중복되는 경우는 모두 제거하기로 함 + index 초기화

In [26]:
#같은 id로 동시에 찍힌 경우 
band_df2 = (band_df[~band_df.duplicated(subset = ['id','dt' ,'bid'])]).reset_index(drop = True)
tag_df2 = (tag_df[~tag_df.duplicated(subset = ['id','dt' ,'bid'])]).reset_index(drop = True)


band_df2

,id,type,dt,bid,temp,hum,co2,voc,dust1,dust2_5,...,hr,walk,kcal,dist,sl_idx,sl_time,wear,st_idx,signal,hour
0,f5:88:ed:00:02:79,b,2020-07-21 00:02:40.796000+09:00,24:0A:C4:4E:A9:10,29.6,61.4,485,96,14,14,...,0,0,0,0,0,0,1,71,-72,0
1,d3:dd:d3:5b:88:c2,b,2020-07-21 00:02:40.796000+09:00,24:0A:C4:4E:A9:10,29.6,61.4,485,96,14,14,...,0,0,0,0,0,0,1,100,-62,0
2,e4:13:7d:7b:d7:0a,b,2020-07-21 00:02:40.796000+09:00,24:0A:C4:4E:A9:10,29.6,61.4,485,96,14,14,...,0,0,0,0,0,0,1,0,-78,0
3,f5:c3:3a:11:23:17,b,2020-07-21 00:02:40.796000+09:00,24:0A:C4:4E:A9:10,29.6,61.4,485,96,14,14,...,0,0,0,0,0,0,1,85,-63,0
4,e4:15:d7:13:40:01,b,2020-07-21 00:02:40.796000+09:00,24:0A:C4:4E:A9:10,29.6,61.4,485,96,14,14,...,0,0,0,0,30,25,1,94,-70,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438938,dd:26:89:fb:6b:70,b,2020-07-22 00:02:42.701000+09:00,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,...,0,0,0,0,0,0,1,0,-75,0
438939,d5:aa:2f:ea:b1:8a,b,2020-07-22 00:02:42.701000+09:00,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,...,0,0,0,0,0,0,1,0,-90,0
438940,cb:d7:d9:4a:31:4f,b,2020-07-22 00:02:42.701000+09:00,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,...,0,0,0,0,0,0,1,100,-76,0
438941,ec:50:2e:b5:cf:c4,b,2020-07-22 00:02:42.701000+09:00,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,...,0,0,0,0,0,0,1,0,-71,0


In [27]:
tag_df2.tail()

,id,type,d,dt,bid,temp,hum,co2,voc,dust1,dust2_5,dust10,hour
1621199,02:11:23:34:5b:85,t,-80,2020-07-22 00:02:42.701000+09:00,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,10,0
1621200,02:11:23:34:5b:39,t,-67,2020-07-22 00:02:42.701000+09:00,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,10,0
1621201,02:11:23:34:5b:cc,t,-68,2020-07-22 00:02:42.701000+09:00,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,10,0
1621202,02:11:23:34:5b:b7,t,-69,2020-07-22 00:02:42.701000+09:00,24:0A:C4:4E:A9:10,29.9,58.6,736,93,10,10,10,0
1621203,dd:34:01:02:f9:3e,t,-93,2020-07-22 00:02:43.654000+09:00,24:0A:C4:4E:A8:B8,29.9,58.6,736,95,9,9,9,0


##### 시각화 용이하게  dt Column의 형태를 변환

In [ ]:
band_df2['dt'] = pd.to_datetime(band_df2['dt'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')))
tag_df2['dt'] = pd.to_datetime(tag_df2['dt'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')))

band_df

##### 처리 완료 , pickle형태로 데이터 저장


In [30]:
band_df2.to_pickle('band_df.pkl')
tag_df2.to_pickle('tag_df.pkl')